In [1]:
# Исследовать структуру html-файлов, чтобы произвести парсинг всех данных. 
# В каждом файле содержится информация об одном или нескольких объектах из случайной предметной области. 
# Перечень всех характеристик объекта может меняться (у отдельного объекта могут отсутствовать некоторые характеристики). 
# Полученные данные собрать и записать в json. 
# Выполните также ряд операций с данными:
# отсортируйте значения по одному из доступных полей
# выполните фильтрацию по другому полю (запишите результат отдельно)
# для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
# для одного текстового поля посчитайте частоту меток.

In [1]:
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd

In [2]:
def handle_file(file_name):
    items = list()
    with open(file_name, encoding = 'utf-8') as file:
        text = ""
        for row in file.readlines():
            text += row
        site = BeautifulSoup(text, 'html.parser')
        products = site.find_all('div', attrs={'class' : 'product-item'})
        #print(len(products))
        for product in products:
            item = dict()
            item['id'] = product.a['data-id']
            item['link'] = product.find_all('a')[1]['href']
            item['img'] = site.find_all('img')[0]['src']
            item['name'] = product.find_all('span')[0].get_text().strip()
            item['price'] = int(product.price.get_text().replace('₽', '').replace(' ', '').strip())
            item['bonus'] = int(product.strong.get_text().replace('+ начислим ', '').replace(' бонусов', '').strip())
            props = product.ul.find_all('li')
            for prop in props:
                item[prop['type']] = prop.get_text().strip()    
            items.append(item)
    return(items)       

In [3]:
items = []
for i in range(1,67):
  file_name = f'./2/{i}.html'
  items += handle_file(file_name)
print(items[1:6])    

[{'id': '41554', 'link': '/product/41554', 'img': '/upload/63797.png', 'name': '7.0" Xiaomi Mi 32GB', 'price': 70461, 'bonus': 704, 'matrix': 'OLED', 'camera': '32 MP'}, {'id': '38094', 'link': '/product/38094', 'img': '/upload/63797.png', 'name': '6.7" HTC 48GB', 'price': 104618, 'bonus': 1046, 'ram': '7 GB', 'sim': '2 SIM', 'resolution': '1920x2160', 'camera': '35 MP', 'acc': '7219 мА * ч'}, {'id': '12068', 'link': '/product/12068', 'img': '/upload/63797.png', 'name': '6.7" Dell 176GB', 'price': 228547, 'bonus': 2285, 'sim': '3 SIM', 'matrix': 'OLED', 'resolution': '1920x2160', 'camera': '61 MP'}, {'id': '74878', 'link': '/product/74878', 'img': '/upload/63797.png', 'name': '7.0" Asus 224GB', 'price': 45457, 'bonus': 454, 'ram': '9 GB', 'sim': '3 SIM', 'matrix': 'AMOLED', 'camera': '123 MP', 'acc': '5519 мА * ч'}, {'id': '46127', 'link': '/product/46127', 'img': '/upload/63797.png', 'name': '5.1" NVIDIA 176GB', 'price': 366661, 'bonus': 3666, 'ram': '4 GB', 'matrix': 'OLED', 'resolut

In [4]:
with open('result_all_63_2.json', 'w', encoding = 'utf-8') as f:
    f.write(json.dumps(items, ensure_ascii=False))

Отсортируeм значения по полю ['price'] в убывающем порядке:

In [5]:
items = sorted(items, key = lambda x: x['price'], reverse = True)
print(items[1:6])

[{'id': '44213', 'link': '/product/44213', 'img': '/upload/720887.png', 'name': '7.0" Kingston 224GB', 'price': 499790, 'bonus': 4997, 'sim': '2 SIM', 'resolution': '1366x768', 'acc': '5937 мА * ч'}, {'id': '19836', 'link': '/product/19836', 'img': '/upload/764550.jpg', 'name': '5.3" ADATA 32GB', 'price': 499627, 'bonus': 4996, 'processor': '6x3.9 ГГц', 'ram': '14 GB', 'sim': '1 SIM', 'camera': '46 MP'}, {'id': '78731', 'link': '/product/78731', 'img': '/upload/246124.png', 'name': '5.8" Nokia 16GB', 'price': 499502, 'bonus': 4995, 'processor': '4x3.0 ГГц', 'sim': '4 SIM', 'matrix': 'OLED'}, {'id': '38708', 'link': '/product/38708', 'img': '/upload/439013.svg', 'name': '5.8" Samsung S21 128GB', 'price': 499494, 'bonus': 4994, 'processor': '8x4.3 ГГц', 'resolution': '1366x768', 'camera': '73 MP'}, {'id': '41356', 'link': '/product/41356', 'img': '/upload/270132.jpg', 'name': '5.8" Transcend 80GB', 'price': 499455, 'bonus': 4994, 'processor': '2x4.1 ГГц', 'ram': '10 GB', 'sim': '2 SIM', 

Выполним фильтрацию по полю ['bonus']:

In [6]:
filtered_items = []
for goods in items:
    if goods['bonus'] > 4960:
        filtered_items.append(goods)
print(filtered_items[1:6])        

[{'id': '44213', 'link': '/product/44213', 'img': '/upload/720887.png', 'name': '7.0" Kingston 224GB', 'price': 499790, 'bonus': 4997, 'sim': '2 SIM', 'resolution': '1366x768', 'acc': '5937 мА * ч'}, {'id': '19836', 'link': '/product/19836', 'img': '/upload/764550.jpg', 'name': '5.3" ADATA 32GB', 'price': 499627, 'bonus': 4996, 'processor': '6x3.9 ГГц', 'ram': '14 GB', 'sim': '1 SIM', 'camera': '46 MP'}, {'id': '78731', 'link': '/product/78731', 'img': '/upload/246124.png', 'name': '5.8" Nokia 16GB', 'price': 499502, 'bonus': 4995, 'processor': '4x3.0 ГГц', 'sim': '4 SIM', 'matrix': 'OLED'}, {'id': '38708', 'link': '/product/38708', 'img': '/upload/439013.svg', 'name': '5.8" Samsung S21 128GB', 'price': 499494, 'bonus': 4994, 'processor': '8x4.3 ГГц', 'resolution': '1366x768', 'camera': '73 MP'}, {'id': '41356', 'link': '/product/41356', 'img': '/upload/270132.jpg', 'name': '5.8" Transcend 80GB', 'price': 499455, 'bonus': 4994, 'processor': '2x4.1 ГГц', 'ram': '10 GB', 'sim': '2 SIM', 

In [7]:
with open("result_filtered_63_2.json", "w", encoding = 'utf-8') as file:
    file.write(json.dumps(filtered_items, ensure_ascii=False))

для числового поля ['price'] посчитаем статистические характеристики (сумма, мин/макс, среднее и т.д.)

In [8]:
stats_items = []
df = pd.DataFrame(items)
pd.set_option('display.float_format', '{:.1f}'.format)
stats = df['price'].agg(['max', 'min', 'mean', 'median', 'std']).to_dict()
stats_items.append(stats)
print(stats_items)

[{'max': 499855.0, 'min': 10009.0, 'mean': 255138.2688253012, 'median': 256149.5, 'std': 143762.4396247343}]


для текстового поля ['img'] посчитайте частоту меток

In [9]:
text_items = []
text = [item['img'] for item in items]
f1 = collections.Counter(text)
text_items.append(f1)
print(text_items)

[Counter({'/upload/525372.png': 99, '/upload/602417.webp': 99, '/upload/150075.png': 98, '/upload/746582.jpeg': 98, '/upload/109315.png': 97, '/upload/662043.jpeg': 97, '/upload/635168.webp': 96, '/upload/720887.png': 95, '/upload/926788.jpg': 92, '/upload/970038.jpg': 92, '/upload/241147.webp': 92, '/upload/790116.png': 92, '/upload/305983.jpeg': 89, '/upload/482637.jpeg': 85, '/upload/360930.jpeg': 82, '/upload/199460.jpg': 80, '/upload/683231.png': 75, '/upload/216920.jpg': 75, '/upload/798640.webp': 74, '/upload/270132.jpg': 72, '/upload/486501.png': 72, '/upload/690694.svg': 72, '/upload/589657.webp': 69, '/upload/260313.png': 69, '/upload/582730.webp': 67, '/upload/957122.jpeg': 67, '/upload/648344.webp': 67, '/upload/489871.png': 66, '/upload/63797.png': 63, '/upload/815128.webp': 61, '/upload/640534.svg': 60, '/upload/491298.png': 59, '/upload/764550.jpg': 58, '/upload/884356.webp': 58, '/upload/439013.svg': 57, '/upload/496726.webp': 57, '/upload/214467.svg': 56, '/upload/8371